# MLP - Multilayer perceptron

In [ ]:
%pylab inline

In [ ]:
class mlp:
    '''Wielowarstwowy perceptron (1 warstwa ukryta)'''
    def __init__(self, n_in, n_hid, n_out):
        
        # liczba neuronów w kolejnych warstwach
        self.n_in, self.n_hid, self.n_out = n_in, n_hid, n_out

        # stany neuronów
        self.S_out=zeros(n_out)         # O_i
        self.S_hid=zeros(n_hid)         # V_j
        self.S_in=zeros(n_in+1)         # ξ_k
        self.S_in[-1]=1                 # bias node

        # pola lokalne
        self.h_hid=zeros(n_hid)         # h_j
        self.h_out=zeros(n_out)         # h_i

        # wagi połączeń
        self.W_ih=zeros((n_in+1,n_hid)) # w_jk - odwrócona kolejność indeksów
        self.W_ho=zeros((n_hid,n_out))  # W_ij - odwrócona kolejność indeksów

        # delty
        self.dW_ih=zeros((n_in+1,n_hid)) # Δw
        self.dW_ho=zeros((n_hid,n_out))  # ΔW
        
        self.reset()
    
    def reset(self):
        self.W_ih = normal(0, 1, self.W_ih.shape)
        self.W_ho = normal(0, 1, self.W_ho.shape)
        
        
        
    def f(self, x):
        '''Funkcja aktywacji'''
#         return 1/(1+exp(-x))  # exp zwraca NaN dla dżych arg.
        return (tanh(x/2)+1)/2  # to samo co wyżej, ale działa

    def Df(self, x):
        '''Pochodna funkcji aktywacji'''
        y=self.f(x)
        return y*(1-y)

    def feed(self, inp):
        '''Przekazuje wektor danych do warstwy wejściowej'''
        self.S_in[:self.n_in]=inp

    def forward(self):
        '''Propaguje sygnał poprzez kolejne warstwy sieci'''
        self.h_hid = self.S_in @ self.W_ih     # h_j
        self.S_hid = self.f(self.h_hid)        # g(h_j)
        self.h_out = self.S_hid @ self.W_ho    # h_i
        self.S_out = self.f(self.h_out)        # g(h_i)
        
    def diff(self, p):
        return self.trainData[p,self.n_in:] - self.S_out

    def error(self,p):
        return sum(self.diff(p)**2)/2
    
    def setTrainData(self,data):
        '''Zapamiętuje referencję do wektorów uczących'''
        self.trainData=data

    def eval(self,inp):
        self.feed(inp)
        self.forward()
        return self.S_out

    def train(self, eta):
        '''Jedna epoka.'''
    
        Er=0
        p_num=self.trainData.shape[0]

        self.dW_ih.fill(0)
        self.dW_ho.fill(0)

        for p in range(p_num):
            self.feed(self.trainData[p,:self.n_in])
            self.forward()
        
            self.delta_out = self.Df(self.h_out)*self.diff(p)                   # δ_i
            self.dW_ho += outer(self.S_hid, self.delta_out)                     # ΔW_ij
            self.delta_hid = self.Df(self.h_hid)*(self.W_ho @ self.delta_out)   # δ_j
            self.dW_ih += outer(self.S_in, self.delta_hid)                      # Δw_jk

            Er+=self.error(p)

        self.W_ih+=eta*self.dW_ih
        self.W_ho+=eta*self.dW_ho
        return Er   

In [ ]:
# XOR
# wzorce postaci [in_1, in_2, out_1]
pat=array([[0,0,0],[0,1,1],[1,0,1],[1,1,0]])
m=mlp(2, 5, 1)
m.setTrainData(pat)

In [ ]:
m.reset()
hist=[]

In [ ]:
eta=1
for epoch in range(1000):
    hist.append(m.train(eta))
print(hist[-1])

In [ ]:
plot(hist)

In [ ]:
n=10 # rozdzielczość
z=zeros((n,n))
x=linspace(0,1,n)
for i in range(n):
    for j in range(n):
        z[i,j]=m.eval([x[j],x[i]])
figsize(5,5)
contourf(x,x,z, vmin=0, vmax=1);

In [ ]:
m.eval([0,0]), m.eval([0,1]), m.eval([1,0]), m.eval([1,1])